In [20]:
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    %cd "/content/gdrive/My Drive/Projects"
    !git clone https://github.com/ai357060/modelF.git
    import sys
    root_dir="/content/gdrive/My Drive/Projects/modelF"
    sys.path.append(root_dir)
    %cd "/content/gdrive/My Drive/Projects/modelF/wm"
    !git fetch --all
    !git reset --hard origin/master

In [21]:
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    %cd "/content/gdrive/My Drive/Projects"
#     !git clone https://github.com/ai357060/modelF.git
    import sys
    root_dir="/content/gdrive/My Drive/Projects/modelF"
    sys.path.append(root_dir)
    %cd "/content/gdrive/My Drive/Projects/modelF/wm"
#     !git pull

In [22]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [23]:
import pandas as pd
import numpy as np
import datetime as dt
import importlib
import tradestats
import pandas as pd
from datetime import datetime

In [24]:
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
import graphviz

In [25]:
importlib.reload(tradestats)
from tradestats import *

In [26]:
prices = loaddata_4h('AUDUSD_Candlestick_4_Hour_BID_03.08.2003-04.09.2021_EEF_GMT.csv')

In [27]:
rsiKey = [13]
tdiKey = [13]
dictList = []
colFeat = []
keyList = []
# rsiDict   = rsi(prices, rsiKey)
tdiDict   = tdi1(prices, tdiKey)
# dictList.append(rsiDict.df); colFeat.append('rsi'); keyList.append(rsiKey) 
dictList.append(tdiDict.df); colFeat.append('tdi'); keyList.append(tdiKey) 

In [28]:
# prices.dropna(inplace=True)
masterFrame = pd.DataFrame(index=prices.index)
masterFrame[['id','year','date','hour','weekday','open','high','low','close','volume']] = prices[['id','year','date','hour','weekday','open','high','low','close','volume']]   

In [29]:
for i in range(0,len(dictList)):#iteracja po funkcjach
    for j in keyList[i]: #iteracja po okresach
        for k in list(dictList[i][j]): #iteracja po kolumnach danej funkcji w okresie
            colID = colFeat[i] + str(j) + k
            masterFrame[colID] = dictList[i][j][k]

In [30]:
# masterFrame.to_csv(sep=';',path_or_buf='../Data/masterFrame.csv',date_format="%Y-%m-%d",index = False,na_rep='')

In [31]:
masterFrame = masterFrame.dropna()

In [32]:
# masterFrame.iloc[0:-50,:20]

In [33]:
importlib.reload(tradestats)
from tradestats import *

In [34]:
starttime = datetime.now()
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5],[13],[0.003],False)
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5,9],[13,17],[0.001,0.002,0.003,0.004,0.005,0.006],True)
alltrades = preparetrades(masterFrame,[1],[5],[13,17],[0.001,0.002,0.003])
trades = cleartrades(alltrades.copy(),False)
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5],[13],[0.002],True)
endtime = datetime.now()
print(str(endtime - starttime))

0:00:04.551204


In [ ]:
#____________________________________________________________________________________________________________________
#
#
trades.head(2)

In [ ]:
X = trades.iloc[:, :-1].values
feature_names = trades.columns[:-1]
y = trades.iloc[:, -1:].values

In [ ]:
# regr = DecisionTreeRegressor(min_samples_leaf=0.001,criterion='mse',max_depth=17, random_state=1234)
# criterion{“mse”, “friedman_mse”, “mae”, “poisson”}, default=”mse”
regr = DecisionTreeClassifier(min_samples_leaf=1,criterion='gini',max_depth=8, random_state=1234)
# criterion{“gini”, “entropy”}, default=”gini”
regr.fit(X, y)
score = regr.score(X, y)
print(score)

In [ ]:
"""
all features mse depth 8 leaf 0.05 =0,0049
wyciąłem poniższe mse depth 8 leaf 0.05 =0,0043
#     df['green1_red1'] = df.green1 - df.red1
#     df['green2_red2'] = df.green2 - df.red2
#     df['green_red_change'] = df.green1_red1 - df.green2_red2
#     df['green_red_slope_change'] = df.green_slope - df.red_slope
all features mse depth 8 leaf 0.1 =0.0028
all features friedman_mse depth 8 leaf 0.05 =0.0049
all features mse depth 8 leaf 0.03 = 0.0069
all features mse depth 8 leaf 0.01 = 0.0074
all features mse depth 8 leaf 1 = 0.048
all features mse depth 9 leaf 1 = 0.051
all features mse depth 15 leaf 1 = 0.11
all features mse depth 15 leaf 0.001 = 0.054
all features mse depth 15 leaf 0.05 = 0,028
all features mse depth 5 leaf 0.05 = 0.0016
"""

In [ ]:
dd = pd.DataFrame()
dd['f'] = feature_names.values
dd['i'] = regr.feature_importances_
dd.sort_values(by=['i'], ascending=False)

In [ ]:
# text_representation = tree.export_text(regr)
# print(text_representation)

In [ ]:
from graphviz import Source
from sklearn import tree
graph = Source( tree.export_graphviz(regr, out_file=None, feature_names=feature_names,rounded = True, proportion = False, 
                 filled = True))
graph.format = 'png'
graph.render('dtree_render',view=True)

In [ ]:
alltrades.head()

In [ ]:
importlib.reload(tradestats)
from tradestats import *
stats = stattrades(alltrades)

In [ ]:
stats.sort_values("profit_ratio",ascending=False)

In [39]:
importlib.reload(tradestats)
from tradestats import *

tradetypes = [1]
openhours = [5]
closehours = [13]
sls = [10] #,20,30,40,50,60
bar2froms = [-1000,-20,-10,0,10,20,1000]
bar2tos = [-1000,-20,-10,0,10,20,1000]
bar1froms = [-1000,-20,-10,0,10,20,1000]
bar1tos =[-1000,-20,-10,0,10,20,1000]
bar2froms = [-1000,1000]
bar2tos = [-1000,1000]
bar1froms = [-1000,1000]
bar1tos =[-1000,1000]
gr2froms = [-1000,-8,-3,0,3,8,1000]
gr2tos = [-1000,-8,-3,0,3,8,1000]
gr1froms = [-1000,-8,-3,0,3,8,1000]
gr1tos = [-1000,-8,-3,0,3,8,1000]
starttime = datetime.now()
stats,stats0 = stathyper(alltrades,tradetypes,openhours,closehours,sls,bar2froms,bar2tos,bar1froms,bar1tos,gr2froms,gr2tos,gr1froms,gr1tos)
endtime = datetime.now()
print(str(endtime - starttime))

tradetype 1
  openhour 5
    closehour 13
      sl 10
        bar2from -1000
          bar2to -1000
          bar2to 1000
            bar1from -1000
              bar1to -1000
              bar1to 1000
            bar1from 1000
              bar1to -1000
              bar1to 1000
        bar2from 1000
          bar2to -1000
          bar2to 1000
0:00:21.472626


In [38]:
stats.sort_values("profit_sum",ascending=False)

,tradetype,openhour,closehour,sl,bar2from,bar2to,bar1from,bar1to,gr2from,gr2to,gr1from,gr1to,count,countup,countdown,mean,profit_sum,maxdown,profit_ratio,maxdown_ratio
0,1,5,13,10,-1000,1000,-1000,1000,-1000,1000,0,8,2862,1569,1293,0.509224,1457.4,-120.9,145.74,-12.09
0,1,5,13,10,-1000,1000,-1000,1000,-8,1000,0,8,2859,1568,1291,0.499091,1426.9,-120.9,142.69,-12.09
0,1,5,13,10,-1000,1000,-1000,1000,-1000,1000,0,3,1787,1207,580,0.792222,1415.7,-70.0,141.57,-7
0,1,5,13,10,-1000,1000,-1000,1000,-8,1000,0,3,1784,1206,578,0.776457,1385.2,-70.0,138.52,-7
0,1,5,13,10,-1000,1000,-1000,1000,-3,1000,0,8,2773,1541,1232,0.460043,1275.7,-110.9,127.57,-11.09
0,1,5,13,10,-1000,1000,-1000,1000,-3,1000,0,3,1711,1182,529,0.712916,1219.8,-50.0,121.98,-5
0,1,5,13,10,-1000,1000,-1000,1000,-1000,8,0,3,1777,1202,575,0.681148,1210.4,-70.0,121.04,-7
0,1,5,13,10,-1000,1000,-1000,1000,-8,8,0,3,1774,1201,573,0.665107,1179.9,-70.0,117.99,-7
0,1,5,13,10,-1000,1000,-1000,1000,0,1000,0,8,2167,1135,1032,0.537886,1165.6,-116.2,116.56,-11.62
0,1,5,13,10,-1000,1000,-1000,1000,-1000,3,0,1000,1777,1129,648,0.612606,1088.6,-70.0,108.86,-7


In [ ]:
stats.to_csv(sep=';',path_or_buf='../Data/stats10.csv',date_format="%Y-%m-%d",index = False,na_rep='')